<h1 style='background:#3E4491; border:2; border-radius:5px; font-size:200%; font-weight: bold;
color:white; padding:20px'><center>Estudo TG2 - Técnicas de Detecção de Fraudes usando Sistemas Inteligentes Híbridos</center></h1>

<center><img src = "Fraud_Detect.jpg" width = 900 height = 400/></center>

* Dataset (https://www.kaggle.com/datasets/chitwanmanchanda/fraudulent-transactions-data):

    Dados fictícios que simulam 6.5 milhões de transações financeiras em sistema de carteira digital mobile, em que ~0,13% são fraudes. Trata-se de problema de aprendizado supervisionado (labeled data). Dados gerados pelo software Paysim, criado por Edgar Lopez e encontrado em https://github.com/EdgarLopezPhD/PaySim

* Tarefa:
Estudar técnicas diversas de aprendizado de máquina para detecção de fraudes financeiras, com foco Stacking Ensembles.

<a id='top'></a>
<div class="list-group" id="list-tab" role="tablist">

<h1 style='background:#3E4491; border:0; border-radius: 10px; color:white;padding:20px'><center> Sumário </center></h1>

### [**1. Importando bibliotecas, carregando dados e Data Wrangling**](#title-one)

### [**2. EDA e Seleção de Features**](#title-two)

### [**3. Limpeza de dados**](#title-three)

### [**4. Pré-processamento de Dados**](#title-four)

### [**5. Modelagem**](#title-five)

### [**6. Pickle e Simulação de Deploy**](#title-six)

<a id="title-one"></a>
<h1 style='background:#3E4491; border:2; border-radius: 10px; color:white;padding:20px'><center>Importando bibliotecas, carregando dados e Data Wrangling</center></h1>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

sns.set()
sns.color_palette('hls', 8)

# Definindo fonte
font = {'family': 'DejaVu Sans',
        'weight': 'bold',
        'size': 30
        }

# Ignorando warnings
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

### Carregando dados

In [2]:
fraud_df = pd.read_csv('Fraud.csv')
fraud_df_orig = fraud_df.copy()

fraud_df.info()

In [ ]:
fraud_df.sample(10, random_state = 51)

#### - Colunas:

* step - maps a unit of time in the real world. In this case 1 step is 1 hour of time. Total steps 744 (30 days simulation).

* type - CASH-IN, CASH-OUT, DEBIT, PAYMENT and TRANSFER.

* amount - amount of the transaction in local currency.

* nameOrig - customer who started the transaction

* oldbalanceOrg - initial balance before the transaction

* newbalanceOrig - new balance after the transaction

* nameDest - customer who is the recipient of the transaction

* oldbalanceDest - initial balance recipient before the transaction. Note that there is not information for customers that start with M (Merchants).

* newbalanceDest - new balance recipient after the transaction. Note that there is not information for customers that start with M (Merchants).

* isFraud - This is the transactions made by the fraudulent agents inside the simulation. In this specific dataset the fraudulent behavior of the agents aims to profit by taking control or customers accounts and try to empty the funds by transferring to another account and then cashing out of the system.

* isFlaggedFraud - The business model aims to control massive transfers from one account to another and flags illegal attempts. An illegal attempt in this dataset is an attempt to transfer more than 200.000 in a single transaction.

### Balanceando o Dataset (Undersampling)

In [ ]:
frauds = fraud_df[fraud_df['isFraud'] == 1]
legits = fraud_df[fraud_df['isFraud'] == 0]

n_legits = len(legits)
n_frauds = len(frauds)

legits_percent = (n_legits / (n_frauds + n_legits)) * 100
frauds_percent = (n_frauds / (n_frauds + n_legits)) * 100

print("Número de transações legítimas: ", n_legits, '({:.4f}%).'.format(legits_percent))
print("Número de transações fraudulentas: ", n_frauds, '({:.4f}%).'.format(frauds_percent))

#Pegando 80% do total de transações fraudulentas para melhorar a performance do modelo de deploy
frauds = fraud_df[fraud_df['isFraud'] == 1].sample(frac = 0.82, random_state = 42)
n_frauds = len(frauds) #resetando valor de n_frauds para a mudança anterior

legits = legits.sample(n = n_frauds, random_state = 42)
df_reduced = pd.concat([frauds, legits], axis = 0)

fraud_value_counts = df_reduced.isFraud.value_counts(normalize = True)
print(f'\nFazendo o Undersampling, temos agora {fraud_value_counts[0]*100}% de transações legítimas e {fraud_value_counts[1]*100}% de transações fraudulentas.')

<a id="title-two"></a>
<h1 style='background:#3E4491; border:2; border-radius: 10px; color:white;padding:20px'><center>EDA e Seleção de Features</center></h1>

In [ ]:
df_reduced.head()

In [ ]:
# Lista das variáveis numéricas de ponto flutuante
numericals = ['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']

df_reduced[numericals].describe()

In [ ]:
# Importando biblioteca p/ calcular o VIF (Variance Inflation Factor)

from statsmodels.stats.outliers_influence import variance_inflation_factor

def calc_vif(df):

    # Calculando VIF
    vif = pd.DataFrame()
    vif["variables"] = df.columns
    vif["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]

    return(vif)

calc_vif(df_reduced.drop(['step', 'type', 'nameOrig', 'nameDest'], axis = 1))

In [ ]:
# # Podemos ver que Saldo Anterior na Origem (*oldbalanceOrg*) e Saldo Atual na Origem (*newbalanceOrig*) tem altíssimos VIF's, assim mostrando que estão altamente correlacionados.
# # O mesmo ocorre para Saldo Anterior no Destino (*oldbalanceDest*) e Saldo Atual no Destino (*newbalanceDest*). Vamos juntar esse par de variáveis colineares e *dropar* elas individualmente.

# df_reduced['actualAmountOrig'] = df_reduced.apply(lambda x: x['oldbalanceOrg'] - x['newbalanceOrig'], axis = 1)
# df_reduced['actualAmountDest'] = df_reduced.apply(lambda x: x['oldbalanceDest'] - x['newbalanceDest'], axis = 1)

# # Dropando colunas individuais
# df_reduced = df_reduced.drop(['oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest'],axis = 1)

# # Recalculando VIF
# calc_vif(df_reduced.drop(['step', 'type', 'nameOrig', 'nameDest'], axis = 1))

In [ ]:
plt.figure(figsize=(20,18))

corr = df_reduced.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap='PuOr', vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5},
            linecolor='k', fmt='.2f', annot=True)
plt.show()

In [ ]:
nome_colunas = {'amount': 'Montante',
                'oldbalanceOrg': 'Saldo Anterior - Origem',
                'newbalanceOrig': 'Saldo Atual - Origem',
                'oldbalanceDest': 'Saldo Anterior - Destino',
                'newbalanceDest': 'Saldo Atual - Destino',
                'step': 'Passo - Tempo',
                'isFraud': 'Fraude',
                'isFlaggedFraud': 'Flag de Fraude',
                }

fig = plt.figure(figsize= (15,9))

num_cols = list(df_reduced.select_dtypes(exclude=['object','int64']).columns)

for i, col in enumerate(num_cols):
    ax=fig.add_subplot(len(nome_colunas)-3,1,i+1)
    sns.boxplot(x=df_reduced[col], ax=ax)
    plt.xlabel(nome_colunas[col], fontdict={'fontsize': 15})
    fig.tight_layout()

fig.suptitle('Distribuição de variáveis - Sem limpeza de Intervalo Interquartil\n', size = 24)
plt.subplots_adjust(top=0.90)
plt.show()

In [ ]:
fig, axs = plt.subplots(len(nome_colunas)-3, 1, figsize = (12,18))

for i, col in enumerate(numericals):
    sns.histplot(data = df_reduced, x = col, hue = 'isFraud', ax = axs[i], bins=40)
    axs[i].set_xlabel(nome_colunas[col], fontdict={'fontsize': 15})
    axs[i].set_ylabel('Frequência', fontdict={'fontsize': 15})
    axs[i].legend(['Fraude', 'Legítima'])

fig.tight_layout()
fig.subplots_adjust(top=0.95)
fig.suptitle('Histograma das variáveis numéricas - Antes da limpeza', fontsize=29);

In [ ]:
# from bokeh.plotting import figure, output_notebook, show, output_file
# from bokeh.models import ColumnDataSource

# p = figure(plot_width = 300, plot_height = 300, tools = "pan,reset,save")
# source = ColumnDataSource(data = dict(x = sample1.amount, y = sample1.index))

# p.circle(x='x', y = 'y', source=source, color = 'navy', alpha = 0.5)

# output_notebook()
# show(p)

As features selecionadas são: Montante, Saldo Anterior - Origem, Saldo Atual - Origem, Saldo Anterior - Destino, Saldo Atual - Destino e **Fraude (variável alvo)**. As variáveis *Step*, nome das contas e *flag* de fraude foram *dropadas* pois não representavam nenhum poder preditivo e poderiam atrapalhar o treinamento.

<a id="title-three"></a>
<h1 style='background:#3E4491; border:2; border-radius: 10px; color:white;padding:20px'><center>Limpeza de Dados</center></h1>

# Data Cleaning

In [ ]:
# from dataprep.eda import plot

# plot(df_reduced)

In [ ]:
df_clean = df_reduced.copy()

print('Número de linhas pré limpeza: ', df_reduced.shape[0])

# Limpando as colunas numéricas (fazendo somente limite superior pois muitos valores nulos - contas sem dinheiro)

for i in ['amount', 'oldbalanceOrg', 'oldbalanceDest', 'newbalanceDest']: #numericals dá erro porque newbalanceOrig tem praticamente todos seus valores nulos assim dropnando todo o dataset
    
    q1, q3 = np.percentile(df_clean[i], [25, 75])
    
    iqr = q3 - q1
    lim_sup = q3 + 1.75 * iqr

    indice = df_clean[df_clean[i] >= lim_sup].index

    df_clean.drop(indice, inplace= True)

print('Número de linhas pós limpeza: ', df_clean.shape[0])

In [ ]:
fig = plt.figure(figsize= (15,9))

for i, col in enumerate(num_cols):
    ax=fig.add_subplot(len(nome_colunas)-3,1,i+1)      
    sns.boxplot(x = df_clean[col], ax=ax)
    plt.xlabel(nome_colunas[col], fontdict={'fontsize': 15})
    fig.tight_layout()

fig.suptitle('Distribuição de variáveis - Após limpeza de Intervalo Interquartil\n', size = 24)
plt.subplots_adjust(top=0.90)
plt.show()

In [ ]:
fig, axs = plt.subplots(len(nome_colunas)-3, 1, figsize = (12,18))

for i, col in enumerate(numericals):
    sns.histplot(data = df_clean, x = col, hue = 'isFraud', ax = axs[i], bins=40)
    axs[i].set_xlabel(nome_colunas[col], fontdict={'fontsize': 15})
    axs[i].set_ylabel('Frequência', fontdict={'fontsize': 15})
    axs[i].legend(['Fraude', 'Legítima'])

fig.tight_layout()
fig.subplots_adjust(top=0.95)
fig.suptitle('Histograma das variáveis numéricas - Após limpeza', fontsize=29);

<a id="title-four"></a>
<h1 style='background:#3E4491; border:2; border-radius: 10px; color:white;padding:20px'><center>Pré-processamento de Dados</center></h1>

In [ ]:
#Imports de Modelagem (Pré-processamento: Normalização, Splits; Validação; e Modelos)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, recall_score, f1_score, classification_report, roc_curve, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay

# Modelos
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, StackingClassifier, IsolationForest
from xgboost import XGBClassifier

In [ ]:
#Normalização das variáveis numéricas

df_clean[numericals] = StandardScaler().fit_transform(df_clean[numericals])

# Codificação da coluna 'Type' (categórica)

codec = pd.DataFrame(df_reduced['type'].factorize()[1], columns = ['type'])
print('* Codificação da variável categórica "type":  \n\n', codec)

df_clean['type'] = df_clean['type'].factorize()[0]

In [ ]:
#Definição de Features e Target

features = ['type', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']

X = df_clean[features]
y = df_clean['isFraud']

#Splits

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify = y, random_state = 42)

<a id="title-five"></a>
<h1 style='background:#3E4491; border:2; border-radius: 10px; color:white;padding:20px'><center>Modelagem</center></h1>

### Testando Modelos com cross_val_score

In [ ]:
# Criando comparação de acurácia entre os 6 modelos escolhidos

comparacao_modelos = pd.DataFrame(columns = ['Modelo', 'Score'])

modelos = [LogisticRegression('l2', solver='lbfgs', random_state=42),
           KNeighborsClassifier(n_neighbors = 41, p = 4),
           ExtraTreesClassifier(random_state = 42),
           IsolationForest(n_jobs = -1, random_state = 42),
           XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42, n_jobs=-1),
           StackingClassifier(estimators = [('xgbc', XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42, n_jobs=-1)),
                                           ('log_reg', LogisticRegression(n_jobs = -1, random_state=42))
                                           ],
                              final_estimator=LogisticRegression('l2', solver='lbfgs', random_state=42)
                              )
           ]

for model in modelos:
    model_name = model.__class__.__name__
    scores = cross_val_score(model, X_train, y_train, cv = 5, scoring = 'accuracy')
    comparacao_modelos.loc[len(comparacao_modelos)] = [model_name, scores.mean().round(4)]

comparacao_modelos = comparacao_modelos.sort_values(by='Score', ascending=False)

print('* Comparação Cross-Validation Score entre modelos: ')
comparacao_modelos

### * Modelo Baseline: Dummy Classifier

In [ ]:
# Definindo objeto do modelo e seus parâmetros

dummymodel = DummyClassifier(strategy = 'prior')
dummymodel.fit(X_train, y_train) # Treinando modelo

y_pred_dummy = dummymodel.predict(X_test) # Criando lista das predições

In [ ]:
# Métricas e Matriz de Confusão

print('Acurácia: ', accuracy_score(y_test, y_pred_dummy).round(4))

sns.set_theme(style = 'white')
cmd1 = ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred_dummy), display_labels = ['Normal', 'Fraude'])
fig, ax = plt.subplots(figsize = (8,8))
plt.grid(False)
cmd1.plot(ax = ax)
plt.rc('font', **font)
plt.title('Matriz de confusão - Dummy Classifier', fontsize = 20)
plt.show()

### * 1º Modelo: Extra Trees Classifier

In [ ]:
# Definindo objeto do modelo e seus parâmetros constantes

xtc = ExtraTreesClassifier(warm_start = True, n_jobs = -1, random_state = 42)

In [ ]:
# Definindo espaço de busca dos hiperparâmetros e executando otimização Bayesiana destes
from skopt.space import Real, Integer, Categorical
from skopt import BayesSearchCV

xtc_param_dict = {'n_estimators': Integer(2, 800),
                  'criterion': Categorical(['gini', 'entropy']),
                  'max_depth': Integer(1, 15)
                  }

bayessearch_xtc = BayesSearchCV(xtc,
                                search_spaces = xtc_param_dict,
                                scoring = 'f1',
                                cv = 3,
                                refit = 'recall',
                                # verbose = 1,
                                n_jobs = -1,
                                random_state = 42
                                ).fit(X_train, y_train)

In [ ]:
# Imprimindo resultados: Melhores parâmetros, melhor score, report de métricas e matriz confusão

print('\nMelhores Parâmetros: ', bayessearch_xtc.best_params_,
      '\nMelhor Score: ', bayessearch_xtc.best_score_,
      '\n', classification_report(y_test, bayessearch_xtc.predict(X_test))
      )

xtc_cm = ConfusionMatrixDisplay(confusion_matrix(y_test, bayessearch_xtc.predict(X_test)),
                                display_labels = ['Normal', 'Fraude']
                                ).plot()

#### Feature Importance (a título de curiosidade)

In [ ]:
# Lista para renomear as features na visualização
feats_ptbr = ['Tipo de Transação',
              'Montante',
              'Saldo Anterior - Origem',
              'Saldo Atual - Origem',
              'Saldo Anterior - Destino',
              'Saldo Atual - Destino'
              ]

# Chamando método de importâncias das features para o modelo ExtraTreesClassifier
feat_importance = pd.Series(xtc.fit(X_train, y_train).feature_importances_,
                            index=feats_ptbr
                            ).sort_values(ascending=False)

plt.figure(figsize = (20,8))
feat_importance.plot(kind = 'bar', color = '#3E4491')
plt.grid(which = 'major', axis = 'y', linestyle = '--')
plt.xticks(fontsize = 19);

### * 2º Modelo: XGBoost Classifier

In [ ]:
# Definindo objeto do modelo e seus parâmetros constantes

xgbc = XGBClassifier(eval_metric = 'logloss',
                     objective = 'binary:logistic',
                     use_label_encoder = False,
                     n_jobs = -1
                     )

In [ ]:
# Definindo espaço de busca dos hiperparâmetros e executando otimização Bayesiana destes

xgbc_param_dict = {'learning_rate': Real(0.01, 0.99),
                   'max_depth': Integer(3,8),
                   'max_leaves': Integer(10, 100),
                  'n_estimators': Integer(1, 1000)
                   }

bayessearch_xgbc = BayesSearchCV(xgbc,
                                 search_spaces = xgbc_param_dict,
                                 scoring = 'recall',
                                 cv = 3,
                                 refit = 'recall',
                                #  verbose = 1,
                                 n_jobs = -1,
                                 random_state = 42
                                 ).fit(X_train, y_train)

In [ ]:
# Imprimindo resultados: Melhores parâmetros, melhor score, report de métricas e matriz confusão

print('\nMelhores Parâmetros: ', bayessearch_xgbc.best_params_,
      '\n Melhor Score: ', bayessearch_xgbc.best_score_, '\n',
      classification_report(y_test, bayessearch_xgbc.predict(X_test))
      )
xgbc_cm = ConfusionMatrixDisplay(confusion_matrix(y_test, bayessearch_xgbc.predict(X_test)), display_labels = ['Normal', 'Fraude']).plot()

### * 3º Modelo: Isolation Forest Classifier

In [ ]:
# Definindo objeto do modelo e seus parâmetros constantes

ifc = IsolationForest(n_estimators = 7,
                      max_samples = 0.75,
                      contamination = 0.5,
                      max_features = 1,
                      n_jobs = -1,
                      warm_start = True,
                      random_state = 42
                      ).fit(X_train.values)

In [ ]:
# Gambiarra para converter o resultado do modelo para 0 e 1, como a saída do predict dos outros modelos
# mapeando -1 para 1 e 1 para 0, método numpy

numpy_map = np.vectorize(lambda x: 1 if x == -1 else 0)

ifc_y_pred = numpy_map(ifc.predict(X_test.values))

In [ ]:
# Imprimindo resultados: report de métricas e matriz confusão

print(classification_report(y_test, ifc_y_pred))
ifc_cm = ConfusionMatrixDisplay(confusion_matrix(y_test, ifc_y_pred), display_labels = ['Normal', 'Fraude']).plot()

### Tentativa de fazer Isolation Forest funcionar como preditor padrão e otimizar via Bayes

In [ ]:
# ifc_param_dict = {'n_estimators': Integer(2, 1000),
#                   'bootstrap': Categorical([True, False]),
#                   'max_features': Integer(1, 6)
#                   }

# bayessearch_ifc = BayesSearchCV(ifc,
#                                 search_spaces = ifc_param_dict,
#                                 scoring = 'recall',
#                                 cv = 3,
#                                 refit = 'recall',
#                                 verbose = 1,
#                                 n_jobs = -1,
#                                 random_state = 42
#                                 ).fit(X_train, y_train)

In [ ]:
# df_ifc = fraud_df.copy()

# df_ifc = df_ifc.drop(['step', 'nameOrig', 'nameDest', 'isFlaggedFraud'], axis=1)

# df_ifc[numericals] = StandardScaler().fit_transform(df_ifc[numericals])
# df_ifc['type'] = df_ifc['type'].factorize()[0]

# X_train_ifc, X_test_ifc, y_train_ifc, y_test_ifc = train_test_split(df_ifc.drop('isFraud', axis=1), df_ifc['isFraud'], test_size=0.3, stratify=df_ifc['isFraud'], random_state=42)

In [ ]:
# # Isolation Forest

# ifc_unbalance = IsolationForest(n_estimators = 100,
#                                 max_samples = 0.4,
#                                 contamination = df_ifc['isFraud'].value_counts(normalize=True)[1]*10,
#                                 max_features = 1,
#                                 n_jobs = -1,
#                                 random_state = 42
#                                 ).fit(X_train_ifc.values)

In [ ]:
# numpy_map = np.vectorize(lambda x: 1 if x == -1 else 0)

# ifc_y_pred_unbalance = numpy_map(ifc.predict(X_test_ifc.values))

In [ ]:
# print(df_ifc['isFraud'].value_counts(normalize=True)[1])

# print(pd.DataFrame(ifc.predict(X_test_ifc.values)).value_counts(normalize=True))

In [ ]:
# print(classification_report(y_test_ifc, ifc_y_pred_unbalance))
# ifc_cm = ConfusionMatrixDisplay(confusion_matrix(y_test_ifc, ifc_y_pred_unbalance), display_labels=['Normal', 'Fraude']).plot()

Resultados muito ruins - mesmo utilizando de todos os 6.5M de dados para treino/teste, fazendo gambiarras para usar otimização bayesiana e como preditor comum, e ainda assim - , modelo pior que o mero chute! Não vale e pena seguir com esse modelo para o Stacking Ensemble. Utilizaremos a Regressão Logística, já que se trata de modelo com relação custo (computacional)-benefício (resultados, métricas etc) das mais altas na classificação, fora ainda a interpretabilidade e simplicidade de explicação.

### * 4º Modelo: Regressão Logística

In [ ]:
# Definindo objeto do modelo e seus parâmetros constantes

log_reg = LogisticRegression(n_jobs = -1, random_state = 42)

In [ ]:
# Definindo espaço de busca dos hiperparâmetros e executando otimização Bayesiana destes

logreg_param_dict = {'penalty': Categorical(['l2', 'none']),
                     'C': Real(0.01, 1.0),
                     'class_weight': Categorical(['balanced', None]),
                     'solver': Categorical(['newton-cg', 'lbfgs', 'sag', 'saga'])
                     }

bayessearch_logreg = BayesSearchCV(log_reg,
                                   search_spaces = logreg_param_dict,
                                   scoring = 'recall',
                                   cv = 3,
                                   refit = 'recall',
                                #    verbose = 1,
                                   n_jobs = -1,
                                   random_state = 42
                                   ).fit(X_train, y_train)

In [ ]:
# Imprimindo resultados: Melhores parâmetros, melhor score, report de métricas e matriz confusão

print('\nMelhores Parâmetros: ', bayessearch_logreg.best_params_,
      '\n Melhor Score: ', bayessearch_logreg.best_score_, '\n',
      classification_report(y_test, bayessearch_logreg.predict(X_test)))

logreg_cm = ConfusionMatrixDisplay(confusion_matrix(y_test, bayessearch_logreg.predict(X_test)), display_labels = ['Normal', 'Fraude']).plot()

### * 5º Modelo: Stacking Classifier

In [ ]:
# Definindo objeto do modelo: modelos base, modelo final e seus respectivos parâmetros constantes

stack = StackingClassifier(estimators = [('xgbc', bayessearch_xgbc.best_estimator_),
                                         ('log_reg', bayessearch_logreg.best_estimator_)
                                         ],
                           final_estimator = LogisticRegression(solver = 'lbfgs', class_weight = 'balanced', random_state = 41, n_jobs = -1, max_iter = 500),
                           n_jobs = -1,
                           passthrough = True
                           ).fit(X_train, y_train)

In [ ]:
# Imprimindo resultados: Acurácia, report de métricas e matriz confusão

print('Acurácia: ', accuracy_score(y_test, stack.predict(X_test)).round(4))
print(classification_report(y_test, stack.predict(X_test)))
stack_cm = ConfusionMatrixDisplay(confusion_matrix(y_test, stack.predict(X_test)), display_labels = ['Normal', 'Fraude']).plot()

## Comparativo Final dos Modelos

### Tabela de Métricas

In [ ]:
comparativo_final_metricas = pd.DataFrame(columns = ['Acurácia', 'Recall', 'FN', 'F1-Score', 'AUROC'])

comparativo_dict = {'Dummy': dummymodel.predict(X_test),
                    'Extra Trees': bayessearch_xtc.predict(X_test),
                    'XGBoost': bayessearch_xgbc.predict(X_test),
                    'Regressão Logística': bayessearch_logreg.predict(X_test),
                    'Stacking': stack.predict(X_test),
                    'Isolation Forest': ifc_y_pred
                    }

for name in comparativo_dict:
    comparativo_final_metricas.loc[name] = [accuracy_score(y_test, comparativo_dict[name]).round(4),
                                            recall_score(y_test, comparativo_dict[name]).round(4),
                                            confusion_matrix(y_test, comparativo_dict[name])[1][0],
                                            f1_score(y_test, comparativo_dict[name]).round(4),
                                            roc_auc_score(y_test, comparativo_dict[name]).round(4)
                                            ]

In [ ]:
comparativo_final_metricas = comparativo_final_metricas.sort_values(by = 'Recall', ascending = False)
comparativo_final_metricas

### Curvas ROC

In [ ]:
# Cálculo de curvas ROC e AUC para cada modelo

d_auc = roc_auc_score(y_test, dummymodel.predict(X_test)) #Dummy AUC
p_auc = roc_auc_score(y_test, y_test) #Perfection AUC
xtc_auc = roc_auc_score(y_test, bayessearch_xtc.predict(X_test)) #ExtraTrees AUC
xgb_auc = roc_auc_score(y_test, bayessearch_xgbc.predict(X_test)) #XGBoost AUC
ifc_auc = roc_auc_score(y_test, ifc_y_pred) #Isolation Forest AUC
reglog_auc = roc_auc_score(y_test, bayessearch_logreg.predict(X_test)) #Regressão Logística AUC
stack_auc = roc_auc_score(y_test, stack.predict(X_test)) #Stacking AUC

d_fpr, d_tpr, _ = roc_curve(y_test, dummymodel.predict(X_test)) #Dummy
p_fpr, p_tpr, _ = roc_curve(y_test, y_test) #Perfection
xtc_fpr, xtc_tpr, _ = roc_curve(y_test, bayessearch_xtc.predict(X_test)) #ExtraTrees
xgb_fpr, xgb_tpr, _ = roc_curve(y_test, bayessearch_xgbc.predict(X_test)) #XGBoost
ifc_fpr, ifc_tpr, _ = roc_curve(y_test, ifc_y_pred) #Isolation Forest
reglof_fpr, reglof_tpr, _ = roc_curve(y_test, bayessearch_logreg.predict(X_test)) #Regressão Logística
stack_fpr, stack_tpr, _ = roc_curve(y_test, stack.predict(X_test)) #Stacking

In [ ]:
# Plotar

sns.set()
fig, ax = plt.subplots(figsize=(12, 12))

plt.plot(d_fpr, d_tpr, linestyle = '--', label = 'Dummy Model (AUROC = %0.3f)' % d_auc)
plt.plot(p_fpr, p_tpr, linestyle = '--', label = 'Perfect Predictions (AUROC = %0.3f)' % p_auc)
plt.plot(xtc_fpr, xtc_tpr, marker = '.', label = 'Extra Trees (AUROC = %0.3f)' % xtc_auc)
plt.plot(xgb_fpr, xgb_tpr, marker = '.', label = 'XGBoost (AUROC = %0.3f)' % xgb_auc)
plt.plot(ifc_fpr, ifc_tpr, marker = '.', label = 'Isolation Forest (AUROC = %0.3f)' % ifc_auc)
plt.plot(stack_fpr, stack_tpr, marker = '.', label = 'Stacking (AUROC = %0.3f)' % stack_auc)

# Título
plt.title('Curvas ROC - Modelos', fontsize = 30)

# Rótulo dos eixos
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

# Legenda
plt.legend()

plt.show()

<a id="title-six"></a>
<h1 style='background:#3E4491; border:2; border-radius: 10px; color:white;padding:20px'><center>Pickle e Simulação de Deploy</center></h1>

In [ ]:
# Criando dataframe de deploy (cópia do dataset completo, com as transformações idênticas às de treino)

df_all = fraud_df.copy()
df_all = df_all.drop(['step', 'nameOrig', 'nameDest', 'isFlaggedFraud'],
                     axis = 1
                     )

df_all = df_all.drop(df_clean.index, axis = 0)

df_all[numericals] = StandardScaler().fit_transform(df_all[numericals])
df_all['type'] = df_all['type'].factorize()[0]

X_all = df_all.drop('isFraud', axis = 1)
y_all = df_all['isFraud']

Aproveitando que utilizamos apenas parte dos dados nos treinamentos (graças ao undersampling 99.867% dos dados ficaram de fora), podemos utilizar os dados que não foram vistos pelos modelos e simular o *deploy* do melhor modelo em dados 'novos', assim testando a capacidade de generalização do preditor.

Portanto, primeiro vamos criar uma linearização exportável do modelo. Assim, numa situação comercial, teríamos um arquivo pronto para utilizar em servidor e/ou serviço de computação na nuvem.

* A título de exemplo e para análise comparativa, primeiro vamos fazer o deploy somente com o melhor modelo de Regressão Logística (**sozinha, SEM** o XGBoost do Stacking):

In [ ]:
# Serializando o melhor modelo de Regressão Logística	

import pickle

pickle.dump(bayessearch_logreg.best_estimator_, open('modelo_logreg.pkl', 'wb'))

A seguinte célula representa tudo o que o servidor teria que realizar com dados novos para realizar as predições em campo.

In [ ]:
# Carregando modelo e fazendo predições

pickle_model_logreg = pickle.load(open('modelo_logreg.pkl', 'rb'))

y_pred_deploy_logreg = pickle_model_logreg.predict(X_all)

In [ ]:
# Imprimindo resultados: Acurácia, report de métricas e matriz confusão

print('Acurácia: ', accuracy_score(y_all, y_pred_deploy_logreg).round(4))

print(classification_report(y_all, y_pred_deploy_logreg))

cmp2 = ConfusionMatrixDisplay(confusion_matrix(y_all, y_pred_deploy_logreg), display_labels = ['Normal', 'Fraude'])
fig, ax = plt.subplots(figsize = (10,10))
plt.grid(False)
cmp2.plot(ax = ax)
plt.rc('font', **font)
plt.title('Matriz de confusão - Pickle Regressão Logística (Simulação Deploy)', fontsize = 14)
plt.show()

print('* Porcentagens da matriz de confusão:\n')
logreg_deploy_cm = (100 * confusion_matrix(y_all, y_pred_deploy_logreg, normalize='all'))
for n, i in enumerate(logreg_deploy_cm):
    logreg_deploy_cm[n] = i.astype('float').round(3)
    
print(logreg_deploy_cm)

* ### Deploy com Stacking Classifier

In [ ]:
# Serializando melhor modelo de Stacking (modelo final)

pickle.dump(stack, open('modelo_final.pkl', 'wb'))

A seguinte célula representa tudo o que o servidor teria que realizar com dados novos para realizar as predições em campo.

In [ ]:
# Carregando modelo e fazendo predições

pickle_model = pickle.load(open('modelo_final.pkl', 'rb'))

y_pred_deploy = pickle_model.predict(X_all)

In [ ]:
# Imprimindo resultados: Acurácia, report de métricas e matriz confusão

print('Acurácia: ', accuracy_score(y_all, y_pred_deploy).round(4))

print(classification_report(y_all, y_pred_deploy))
cmp1 = ConfusionMatrixDisplay(confusion_matrix(y_all, y_pred_deploy), display_labels = ['Normal', 'Fraude'])
fig, ax = plt.subplots(figsize = (10,10))
plt.grid(False)
cmp1.plot(ax = ax)
plt.rc('font', **font)
plt.title('Matriz de confusão - Pickle Modelo Final (Simulação Deploy)', fontsize = 15)
plt.show()

print('* Porcentagens da matriz de confusão:\n')
final_cm = (100 * confusion_matrix(y_all, y_pred_deploy, normalize='all')).astype('str')
for n, i in enumerate(final_cm):
    final_cm[n] = i.astype('float').round(3)
    
print(final_cm)

In [ ]:
df_all['isFraud'].value_counts()

In [ ]:
df_all['isFraud'].value_counts(normalize=True)*100

## Fim